# RAP and QEC Gap Visualization for Thesis

Publication-quality figures showing:
- **Delta_RAP**: Minimum gap during RAP protocol with Ep=0 (avoided crossing)
- **Delta_min(Ep)**: Gap between code space and error space with penalty term

This notebook uses the shared `qec_config` module for all platform parameters,
quantum operators, and utility functions.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add parent directory to path for qec_config import
sys.path.insert(0, os.path.dirname(os.getcwd()))

from qec_config import QECConfig, PlotConfig

# Apply publication-quality plot settings
PlotConfig.apply()

print("="*70)
print("RAP + QEC GAP VISUALIZATION FOR THESIS")
print("="*70)

## Initialize Configuration

Load IBM platform parameters from the shared configuration module.

In [ ]:
# Initialize configuration with IBM platform parameters
config = QECConfig(platform='ibm')
config.info()

## Compute Energy Spectra

Compute energy eigenvalues for two cases:
1. **Ep = 0**: Bare RAP protocol showing the avoided crossing gap
2. **Ep = 50 MHz**: With penalty term showing code-to-error separation

In [ ]:
# Case 1: Ep = 0 (bare RAP)
# Case 2: Ep = 50 MHz

Ep_0 = 0
Ep_large = config.Ep_MHz_to_rad(50)  # 50 MHz -> rad/s

print("Loading/computing energy spectra...")

# Use caching system - loads from data/ if available, otherwise computes and saves
energies_no_penalty, idx_no_pen = config.get_or_compute_spectrum(
    Ep=Ep_0, pulse_type='gaussian'
)

energies_with_penalty, idx_with_pen = config.get_or_compute_spectrum(
    Ep=Ep_large, pulse_type='gaussian'
)

print("Done!")

## Calculate Gap Values

In [ ]:
gaps = config.compute_gaps(energies_no_penalty, energies_with_penalty)

print(f"\nGap Analysis:")
print(f"  Delta_RAP = {gaps['delta_rap']:.3f} MHz (at t = {gaps['t_delta_rap']:.2f} us)")
print(f"  Delta_min(Ep=50 MHz) = {gaps['delta_min_ep']:.3f} MHz (at t = {gaps['t_delta_min_ep']:.2f} us)")
print(f"  Ratio Delta_min/Delta_RAP = {gaps['delta_min_ep']/gaps['delta_rap']:.2f}")

## Create Thesis-Quality Figure

Two-panel figure comparing bare RAP (left) and RAP with penalty (right).

In [ ]:
# Convert to plotting units
t_plot = config.t_list * config.TO_TIME_UNITS  # to us

# Convert energies to MHz
E0_no_pen = energies_no_penalty[0] * config.TO_FREQ_UNITS
E1_no_pen = energies_no_penalty[1] * config.TO_FREQ_UNITS
E0_with_pen = energies_with_penalty[0] * config.TO_FREQ_UNITS
E1_with_pen = energies_with_penalty[1] * config.TO_FREQ_UNITS

# Break lines at swap points
E0_no_pen_broken = config.break_at_swaps(E0_no_pen, idx_no_pen[0])
E1_no_pen_broken = config.break_at_swaps(E1_no_pen, idx_no_pen[1])
E0_with_pen_broken = config.break_at_swaps(E0_with_pen, idx_with_pen[0])
E1_with_pen_broken = config.break_at_swaps(E1_with_pen, idx_with_pen[1])

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), dpi=150)

colors_main = PlotConfig.COLORS_LOGICAL

# ========== LEFT PANEL: Ep = 0 ==========
ax1.plot(t_plot, E0_no_pen_broken, color=colors_main[0], linewidth=3.5,
         label=r'$|0_L\rangle$', zorder=10)
ax1.plot(t_plot, E1_no_pen_broken, color=colors_main[1], linewidth=3.5,
         label=r'$|1_L\rangle$', zorder=10)

# Annotate Delta_RAP
gap_array = np.abs(E1_no_pen - E0_no_pen)
idx_min = np.argmin(gap_array)
t_gap = t_plot[idx_min]
E0_at_gap = E0_no_pen[idx_min]
E1_at_gap = E1_no_pen[idx_min]
mid_gap = (E0_at_gap + E1_at_gap) / 2

ax1.annotate('', xy=(t_gap, E1_at_gap), xytext=(t_gap, E0_at_gap),
             arrowprops=dict(arrowstyle='<->', color=PlotConfig.COLOR_GAP_RAP, lw=3,
                           mutation_scale=20))
ax1.text(t_gap + 0.2, mid_gap, r'$\Delta_{\mathrm{RAP}}$',
         fontsize=20, color=PlotConfig.COLOR_GAP_RAP, weight='bold',
         bbox=dict(boxstyle='round,pad=0.4', facecolor='white',
                   edgecolor=PlotConfig.COLOR_GAP_RAP, linewidth=2, alpha=0.95))

ax1.set_xlabel('Time [us]', fontsize=22)
ax1.set_ylabel('Energy [MHz]', fontsize=22)
ax1.set_title(r'Bare RAP', fontsize=22, pad=15)
ax1.tick_params(axis='both', which='major', labelsize=24, width=2, length=8)
ax1.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, zorder=0, color='gray')
ax1.plot([], [], color='black', linewidth=1.2, linestyle='--', alpha=0.9, label='Error states')
ax1.legend(loc='center right', framealpha=0.95, edgecolor='black', fancybox=True, fontsize=18)

# ========== RIGHT PANEL: Ep = 50 MHz ==========
ax2.plot(t_plot, E0_with_pen_broken, color=colors_main[0], linewidth=3.5, zorder=10)
ax2.plot(t_plot, E1_with_pen_broken, color=colors_main[1], linewidth=3.5, zorder=10)

# Plot error states
for i in range(2, config.dim):
    err_arr = energies_with_penalty[i] * config.TO_FREQ_UNITS
    label = 'Error states' if i == 2 else None
    ax2.plot(t_plot, err_arr, color='black', linewidth=1.2, linestyle='--',
             alpha=0.9, label=label, zorder=5)

# Annotate Delta_min(Ep)
t_mid_idx = len(t_plot) // 2
t_mid = t_plot[t_mid_idx]
E_code_top = max(E0_with_pen[t_mid_idx], E1_with_pen[t_mid_idx])
min_error = np.min([energies_with_penalty[i] * config.TO_FREQ_UNITS
                    for i in range(2, config.dim)], axis=0)
E_error_bottom = min_error[t_mid_idx]
mid_QEC = (E_code_top + E_error_bottom) / 2

ax2.annotate('', xy=(t_mid, E_error_bottom - 0.5), xytext=(t_mid, E_code_top + 0.5),
             arrowprops=dict(arrowstyle='<->', color=PlotConfig.COLOR_GAP_QEC, lw=3,
                           mutation_scale=20))
ax2.text(t_mid + 0.2, mid_QEC, r'$\Delta_{\mathrm{min}}(E_p)$',
         fontsize=20, color=PlotConfig.COLOR_GAP_QEC, weight='bold',
         bbox=dict(boxstyle='round,pad=0.4', facecolor='white',
                   edgecolor=PlotConfig.COLOR_GAP_QEC, linewidth=2, alpha=0.95))

ax2.set_xlabel('Time [us]', fontsize=22)
ax2.set_ylabel('Energy [MHz]', fontsize=22)
ax2.set_title(r'RAP with penalty', fontsize=22, pad=15)
ax2.tick_params(axis='both', which='major', labelsize=24, width=2, length=8)
ax2.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, zorder=0, color='gray')

# Add state labels on right side
t_right = t_plot[-1]
error_labels = ['011', '110', '100', '001', '101', '010']
label_data = [
    (E1_with_pen[-1], r'$|1_L\rangle$', 'blue'),
    (E0_with_pen[-1], r'$|0_L\rangle$', 'red'),
]
for idx, (i, lbl) in enumerate(zip(range(2, config.dim), error_labels)):
    err_arr = energies_with_penalty[i] * config.TO_FREQ_UNITS
    label_data.append((err_arr[-1], f'|{lbl}>', 'black'))

label_data.sort(key=lambda x: x[0])
min_spacing = 20
adjusted_y = []
for i, (energy, _, _) in enumerate(label_data):
    if i == 0:
        adjusted_y.append(energy)
    else:
        adjusted_y.append(max(energy, adjusted_y[-1] + min_spacing))

for (energy, label, color), y_pos in zip(label_data, adjusted_y):
    ax2.text(t_right + 0.1, y_pos, label, fontsize=18, va='center', ha='left',
             color=color, alpha=0.8 if color == 'black' else 1.0)

# Styling
for ax in [ax1, ax2]:
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)

plt.tight_layout()

# Save figures
os.makedirs('../figs', exist_ok=True)
plt.savefig('../figs/RAP_QEC_gaps_thesis.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.savefig('../figs/RAP_QEC_gaps_thesis.png', format='png', dpi=300, bbox_inches='tight')

print(f"\nFigures saved:")
print(f"  -> ../figs/RAP_QEC_gaps_thesis.pdf")
print(f"  -> ../figs/RAP_QEC_gaps_thesis.png")
plt.show()

## Energy Spectrum Splitting vs Penalty Energy Ep

Three-panel figure showing how the energy spectrum changes with increasing penalty.

In [ ]:
Ep_values_plot = [0, 10, 50]  # MHz

fig2, axes = plt.subplots(1, 3, figsize=(18, 5))

print(f"Loading/computing energy spectra for Ep values: {Ep_values_plot} MHz")

for idx, Ep_val in enumerate(Ep_values_plot):
    ax = axes[idx]

    Ep_rad = config.Ep_MHz_to_rad(Ep_val)
    
    # Use caching system
    energies_ep, idx_series_ep = config.get_or_compute_spectrum(
        Ep=Ep_rad, pulse_type='gaussian'
    )

    # Convert to MHz
    E0_ep = energies_ep[0] * config.TO_FREQ_UNITS
    E1_ep = energies_ep[1] * config.TO_FREQ_UNITS
    E0_ep_broken = config.break_at_swaps(E0_ep, idx_series_ep[0])
    E1_ep_broken = config.break_at_swaps(E1_ep, idx_series_ep[1])

    # Plot logical states
    ax.plot(t_plot, E0_ep_broken, color=colors_main[0], linewidth=3.5,
            label=r"$|0_L\rangle$", zorder=10)
    ax.plot(t_plot, E1_ep_broken, color=colors_main[1], linewidth=3.5,
            label=r"$|1_L\rangle$", zorder=10)

    # Plot error states
    for i in range(2, config.dim):
        err_arr = energies_ep[i] * config.TO_FREQ_UNITS
        label = "Error states" if i == 2 else None
        ax.plot(t_plot, err_arr, color='gray', linewidth=1.2, linestyle='--',
                alpha=0.9, label=label, zorder=5)

    # Styling
    ax.set_xlabel(r'time [$\mu$s]', fontsize=22)
    if idx == 0:
        ax.set_ylabel('energy [MHz]', fontsize=22)
    ax.set_title(f'$E_p = {Ep_val}$ [MHz]', fontsize=22, pad=15)
    ax.tick_params(axis='both', which='major', labelsize=24, width=2, length=8)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, zorder=0, color='gray')

plt.tight_layout()
plt.suptitle('Three-qubit repetition code [[3,1,3]]: Energy spectrum with stabilizer penalization',
             fontsize=24, y=1.1)

plt.savefig('../figs/energy_spectrum_splitting_Ep_thesis.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.savefig('../figs/energy_spectrum_splitting_Ep_thesis.png', format='png', dpi=300, bbox_inches='tight')

print(f"\nFigures saved:")
print(f"  -> ../figs/energy_spectrum_splitting_Ep_thesis.pdf")
print(f"  -> ../figs/energy_spectrum_splitting_Ep_thesis.png")
plt.show()

## Sinusoidal RAP Pulse Profiles

In [ ]:
# Generate pulse profiles
t_pulse = np.linspace(0, config.T_max, 1000)
t_pulse_us = t_pulse * config.TO_TIME_UNITS

omega_vals = config.omega_sinusoidal(t_pulse) * config.TO_FREQ_UNITS
delta_vals = config.delta_sinusoidal(t_pulse) * config.TO_FREQ_UNITS

# Create figure
fig_pulse, ax_pulse = plt.subplots(1, 1, figsize=(10, 6), dpi=150)

ax_pulse.plot(t_pulse_us, omega_vals, color='#6a0dad', linewidth=3.5,
              label=r'$\Omega(t)$', zorder=10)
ax_pulse.plot(t_pulse_us, delta_vals, color='#228B22', linewidth=3.5,
              label=r'$\delta(t)$', zorder=10)

ax_pulse.set_xlabel(r'Time [$\mu$s]', fontsize=22)
ax_pulse.set_ylabel('Energy [MHz]', fontsize=22)
ax_pulse.set_title('Sinusoidal RAP Pulse Profiles (IBM Platform)', fontsize=22, pad=15)
ax_pulse.tick_params(axis='both', which='major', labelsize=24, width=2, length=8)
ax_pulse.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, zorder=0, color='gray')
ax_pulse.legend(loc='lower right', framealpha=0.95, edgecolor='black',
                fancybox=True, fontsize=18)

for spine in ax_pulse.spines.values():
    spine.set_linewidth(1.5)

plt.tight_layout()

plt.savefig('../figs/sinusoidal_RAP_pulse_profiles_thesis.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.savefig('../figs/sinusoidal_RAP_pulse_profiles_thesis.png', format='png', dpi=300, bbox_inches='tight')

print(f"\nFigures saved:")
print(f"  -> ../figs/sinusoidal_RAP_pulse_profiles_thesis.pdf")
print(f"  -> ../figs/sinusoidal_RAP_pulse_profiles_thesis.png")
print(f"\n  Omega(t) = omega_max * sin(pi*t/T)  [Sinusoidal transverse]")
print(f"  delta(t) = -omega_max * cos(pi*t/T)  [Sinusoidal longitudinal]")
plt.show()